<div style="font-size: 14pt;">Prof. Krzysztof Rybinski</div><br/><br/>
<div style="font-size: 22pt;"><b>Artificial Intelligence course</b></div><br/><br/>
<div style="font-size: 18pt;">LAB 10</div><br/>
<div style="font-size: 18pt;">- RMP dataset</div><br/>
<div style="font-size: 18pt;">- emotion classification</div><br/>
<div style="font-size: 18pt;">- zero-shot classification</div><br/>

In [40]:
import torch

In [41]:
#exetute this cell only in Google Colab
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from transformers import pipeline
import os
import torch
import numpy as np
import pandas as pd
import pickle

In [43]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [44]:
#get index for the maximum score from prediction in model distilbert-base-uncased-emotion
############################################################
def get_max_index(ld):
    lv = [d['score'] for d in ld]
    lvind = lv.index(max(lv))
    return lvind
############################################################

In [45]:
#get label with the maximum score for model distilbert-base-uncased-mnli
############################################################
def get_max_label(ld):
    val = ld['scores']
    val_ind = lv.index(max(lv))
    label = ld['labels'][val_ind]
    return label
############################################################

In [46]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [47]:
rmp = pd.read_csv("https://raw.githubusercontent.com/nindaba/semister-5-ai/main/lab-9/rmp_small.csv") #for local drive

In [48]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [49]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [50]:
#for the final project skip this line
sentences = sentences[0:5000]
labels = labels[0:5000]

In [51]:
#check documentation https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True, device=0)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [52]:
#check emotion prdictions
prediction = classifier(sentences[0:10])

In [53]:
prediction[1], sentences[1]

([{'label': 'sadness', 'score': 0.017645377665758133},
  {'label': 'joy', 'score': 0.2766183614730835},
  {'label': 'love', 'score': 0.0019553741440176964},
  {'label': 'anger', 'score': 0.635624349117279},
  {'label': 'fear', 'score': 0.06618060171604156},
  {'label': 'surprise', 'score': 0.001975938444957137}],
 "He's fun, but he speeds through lectures and lessons like a madman. If you have not taken Spanish before, you definitely will have to study.")

In [54]:
prediction[6], sentences[6]

([{'label': 'sadness', 'score': 0.001150374417193234},
  {'label': 'joy', 'score': 0.4458451569080353},
  {'label': 'love', 'score': 0.5510021448135376},
  {'label': 'anger', 'score': 0.0007522721425630152},
  {'label': 'fear', 'score': 0.0004594257625285536},
  {'label': 'surprise', 'score': 0.0007906428654678166}],
 "A very easy grader. Best advice would be to go to her after class and speak with her to get to know her on a personal level...that'll help you more in the class. She's a sweet professor with rich worldly experiences that often relate to class materials.")

In [ ]:
prediction = classifier(sentences)

In [ ]:
len(prediction)

In [ ]:
 l_max_scores = [get_max_index(val) for val in prediction]

In [ ]:
l_max_scores[0:10]

In [ ]:
prediction[1]

In [ ]:
#convert scores to labels
aaa = l_max_scores
aaa = ["sadness" if x == 0 else x for x in aaa]
aaa = ["joy" if x == 1 else x for x in aaa]
aaa = ["love" if x == 2 else x for x in aaa]
aaa = ["anger" if x == 3 else x for x in aaa]
aaa = ["fear" if x == 4 else x for x in aaa]
aaa = ["surprise" if x == 5 else x for x in aaa]
aaa[0:10]


In [ ]:
df = pd.DataFrame(list(zip(aaa, labels)), columns =['emotion', 'score'])
df

In [ ]:
#calculate crosstabulations
result = pd.crosstab(df["emotion"], df["score"])
result

In [ ]:
#do zero-shot classification
#check documentation https://huggingface.co/typeform/distilbert-base-uncased-mnli
classifier = pipeline("zero-shot-classification", 
                      model = "typeform/distilbert-base-uncased-mnli",
                      device = 0 
)

In [ ]:
l1 = "poor or bad professor teaching skills"
l2 = "unfair or improper grading"
l3 = "poor or bad professor interpersonal skills or qualities"
l4 = "extremely or very difficult tests or exams"
l5 = "harassment or abuse or improper behavior"
l6 = "course or professor review or opinion"
candidate_labels=[l1, l2, l3, l4, l5, l6]

In [ ]:
len(sentences)

In [ ]:
prediction = classifier(sentences, candidate_labels=candidate_labels)

In [ ]:
prediction[90], prediction[101]

In [ ]:
lv = prediction[90]['scores']
lv

In [ ]:
lvind = lv.index(max(lv))
lvind

In [ ]:
l_max_labels = [get_max_label(val) for val in prediction]

In [ ]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

In [ ]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

# Final project

1. Propose new classification, each group proposes its own </br>
2. Choose two zero shot models from https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=en&sort=downloads </br>
3. Calculate predictions for each model on Google Colab</br>
4. Calculate cross-tabulation for each prediction </br>
5. Compare results, are there any differences </br>
6. Write conclusions about difference, speed of computation etc.</br>
7. Write what the results tell us about the student opinions regarding courses and professors </br>
8. Are there any recommendations for universities' management </br>
9. Make a presentation and send together with Jupyter Notbook to rybinski@rybinski.eu by Wednesday, 12 April 2023, 23:59
<br/><br/>
<div style="font-size: 16pt;">Grading rules</div><br/>

1. Project done in student teams <br/>
2. You can get max 4 points <br/>
3. 1 point for the classification and its justification <br/>
4. 1 point for the correct code for each model
5. 1 point for the summary, lessons learned and presentation clarity <br/>


In [ ]:
classifier = pipeline("zero-shot-classification", 
                      model = "facebook/bart-large-mnli",
                      device = 0
)

In [ ]:
predition = classifier(sentences, candidate_labels=candidate_labels)

In [ ]:
#modify if necessary
l_max_scores = [get_max_index(val) for val in prediction]

In [ ]:
#modify if necessary
#convert scores to labels
aaa = l_max_scores
aaa = ["poor or bad professor teaching skills" if x == 0 else x for x in aaa]
aaa = ["unfair or improper grading" if x == 1 else x for x in aaa]
aaa = ["poor or bad professor interpersonal skills or qualities" if x == 2 else x for x in aaa]
aaa = ["extremely or very difficult tests or exams" if x == 3 else x for x in aaa]
aaa = ["harassment or abuse or improper behavior" if x == 4 else x for x in aaa]
aaa = ["course or professor review or opinion" if x == 5 else x for x in aaa]
aaa[0:10]

In [ ]:
df = pd.DataFrame(list(zip(aaa, labels)), columns =['prof_opinion', 'score'])
df

In [ ]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result